In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('notebook')
pd.set_option('display.max_rows', 100)

In [ ]:
USE_ENSEMBLE_WITH_REGIME_T = True

EXPERIMENT_NAME = f"logistic_ensemble_{'with_regime_t' if USE_ENSEMBLE_WITH_REGIME_T else 'no_regime_t'}"
MODEL_NAME = "LogisticRegression"

PROJECT_ROOT = Path.cwd().parent
RUN_REPORT_DIR = PROJECT_ROOT / 'data' / 'reports' / 'supervised_learning' / EXPERIMENT_NAME / MODEL_NAME

CV_METRICS_PATH = RUN_REPORT_DIR / 'cv_results' / 'cv_fold_metrics.csv'
MDA_SCORES_PATH = RUN_REPORT_DIR / 'mda_results' / 'mda_scores.csv'

In [ ]:
if not CV_METRICS_PATH.exists():
    raise FileNotFoundError(f"CV metrics file not found: {CV_METRICS_PATH}")

cv_metrics_df = pd.read_csv(CV_METRICS_PATH)
print(f"Loaded CV metrics for {len(cv_metrics_df)} folds.")
cv_metrics_df

## 2. Analyze Performance Stability Across Folds

In [ ]:
metrics_to_plot = ['log_loss', 'mcc', 'f1_macro', 'accuracy']

fig, axes = plt.subplots(len(metrics_to_plot), 1, figsize=(12, 5 * len(metrics_to_plot)), sharex=True)
fig.suptitle(f'Performance Metrics Across Folds for {MODEL_NAME}', fontsize=16)

for i, metric in enumerate(metrics_to_plot):
    ax = axes[i]
    sns.barplot(x='fold', y=metric, data=cv_metrics_df, ax=ax, palette='viridis')
    mean_val = cv_metrics_df[metric].mean()
    std_val = cv_metrics_df[metric].std()
    ax.axhline(mean_val, color='r', linestyle='--', label=f'Mean: {mean_val:.3f} ± {std_val:.3f}')
    ax.set_title(f'{metric.replace("_", " ").title()}')
    ax.set_xlabel('')
    ax.set_ylabel('Score')
    ax.legend()

axes[-1].set_xlabel('Fold Number')
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

## 3. Load and Analyze Feature Importance (MDA) Stability

In [ ]:
if not MDA_SCORES_PATH.exists():
    raise FileNotFoundError(f"MDA scores file not found: {MDA_SCORES_PATH}")
    
mda_df = pd.read_csv(MDA_SCORES_PATH)
print(f"Loaded MDA scores for {len(mda_df['feature'].unique())} features across {len(mda_df['fold'].unique())} folds.")

In [ ]:
# Aggregate to find the most important features on average
mean_importances = mda_df.groupby('feature')['importance'].mean().sort_values(ascending=False)
top_n = 30 # Show top 30 features
top_features = mean_importances.head(top_n).index.tolist()

df_top_mda = mda_df[mda_df['feature'].isin(top_features)]

fig, ax = plt.subplots(figsize=(12, 10))
sns.boxplot(
    x='importance',
    y='feature',
    data=df_top_mda,
    order=top_features,
    orient='h',
    palette='viridis_r',
    ax=ax
)

ax.axvline(x=0, color='r', linestyle='--', linewidth=1.2, label='Zero Importance')
ax.set_title(f'Ensemble Feature Importance Stability (MDA)', fontsize=16)
ax.set_xlabel('MDA Importance (Increase in LogLoss)')
ax.set_ylabel('Input Feature (Thematic Model Output)')
ax.legend()
plt.tight_layout()
plt.show()